In [ ]:
import os
import torch
import torch_scatter
import torch_sparse
import torchvision
import torch.nn as nn
import torch_geometric as pyg
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import requests

print(f"PyTorch has version {torch.__version__} with cuda {torch.version.cuda}")


: 

In [ ]:
## Import dataset

BASE_URL="https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/${DATASET_NAME}/"

import torch
from torch.utils.data import Dataset
import requests
import numpy as np

class LearningToSimulateDataset(Dataset):
    def __init__(self, dataset_name, base_url="https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/"):
        self.dataset_name = dataset_name
        self.base_url = base_url

        # Download the dataset if it does not exist.
        if not os.path.exists(f"{dataset_name}.npz"):
            print("Downloading dataset...")
            response = requests.get(f"{base_url}{dataset_name}/train_v1.npz")
            open(f"{dataset_name}.npz", "wb").write(response.content)
            print("Dataset downloaded.")

        # Load the dataset.
        with open(f"{dataset_name}.npz", "rb") as f:
            data = np.load(f)["data"]

        # Convert the data to PyTorch tensors.
        self.inputs = torch.from_numpy(data["input"])
        self.targets = torch.from_numpy(data["target"])

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.targets[index]


: 

In [ ]:
url = "https://storage.googleapis.com/learning-to-simulate-complex-physics/Datasets/WaterRamps/test.tfrecord"
response = requests.get(url).content
print(response)
print(response[2])


: 

# Data Preprocessing

In [26]:
def generate_noise(position_seq, noise_std):
    velocity_seq = position_seq[:, 1:] - position_seq[:, :-1]
    time_steps = velocity_seq.size(1)
    velocity_noise = torch.randn_like(velocity_seq) * (noise_std/time_steps ** 0.5)
    

[1 2 3]


In [ ]:

class InteractionNetwork(pyg.nn.MessagePassing):
    def __init__(self, hidden_size, layers=3):
        super().__init__()
        self.lin_edge = torchvision.ops.MLP(hidden_size*3, hidden_size, layers)
        self.lin_node = torchvision.ops.MLP(hidden_size*2, hidden_size,layers)
        return x

    
    def message(self, x_i, x_j, edge_feature):
        x = torch.cat((x_i, x_j, edge_feature), dim=1)
        x = self.lin_edge(x)
        
    def aggregate(self, inputs, index):
        out = torch_sc